<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Attributes" data-toc-modified-id="Attributes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Attributes</a></span><ul class="toc-item"><li><span><a href="#Test:-STALN" data-toc-modified-id="Test:-STALN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Test: STALN</a></span></li><li><span><a href="#Test-POTEK" data-toc-modified-id="Test-POTEK-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Test POTEK</a></span></li><li><span><a href="#Test-VRSTA" data-toc-modified-id="Test-VRSTA-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Test VRSTA</a></span></li><li><span><a href="#Test-TIPTV" data-toc-modified-id="Test-TIPTV-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Test TIPTV</a></span></li><li><span><a href="#Test-STANJE_IM" data-toc-modified-id="Test-STANJE_IM-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Test STANJE_IM</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li><li><span><a href="#Filter" data-toc-modified-id="Filter-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Filter</a></span></li></ul></div>

# Streams (join features by attribute)

In [1]:
import geopandas as gpd
import os

In [2]:
shp_src = "d:\\nejc\\hand_index\\DRSV_HIDRO5_LIN_PV\\DRSV_HIDRO5_LIN_PV.shp"

In [3]:
gdf = gpd.GeoDataFrame.from_file(shp_src)

## Attributes

In [4]:
print(f"Total number of features: {gdf.shape[0]}")

Total number of features: 480314


In [5]:
print(gdf.columns)

Index(['OBJECTID_1', 'ID_REG_HLI', 'HLINPV_ID', 'HLINPV_IM', 'VERZIJA',
       'DATP_ZAC', 'DATP_KON', 'RAZVOJ', 'JEZIK', 'DAT_ZAC', 'DAT_KON',
       'VODE_ID', 'HMZ_ID', 'IME_ID', 'IME', 'VRSTA_ID', 'VRSTA_IME', 'OS_ID',
       'OS_IME', 'VTOK_ID', 'VTOK_IME', 'IZVOR_ID', 'IZVOR_IM', 'STALN_ID',
       'STALN_IME', 'STANJE_ID', 'STANJE_IM', 'POTEK_ID', 'POTEK_IM',
       'TIPTV_ID', 'TIPTV_IM', 'TIPPREH_ID', 'TIPPREH_IM', 'SIRINA_ID',
       'SIRINA_IM', 'VIRP_ID', 'VIRP_IM', 'DVIRP', 'PAS_VZ', 'PREOB_ID',
       'PREOBLIK', 'geometry'],
      dtype='object')


### Test: STALN


In [6]:
gdf.STALN_IME.value_counts()

ni podatka      362655
funkcionalen    117616
občasen             28
stalen              15
Name: STALN_IME, dtype: int64

In [7]:
gdf.STALN_ID.value_counts()

9999    362655
2       117616
3           28
4           15
Name: STALN_ID, dtype: int64

Decision: include all features, because there is too many attributes with "unknown" label `"9999": "ni podatka"`.

### Test POTEK

In [8]:
gdf.POTEK_IM.value_counts()

da - potek je znan    461652
ne - potek ni znan     18662
Name: POTEK_IM, dtype: int64

In [9]:
gdf.POTEK_ID.value_counts()

1    461652
2     18662
Name: POTEK_ID, dtype: int64

Decision: include only ID `"1": "da - potek je znan"`.

Revised: can as well include `"2"` if it also has VRSTA_ID `"2": "prehod"`.

### Test VRSTA

In [10]:
gdf.VRSTA_IME.value_counts()

tekoča voda              384159
prehod                    92391
stoječa voda povezava      3396
akumulacijsko jezero        368
Name: VRSTA_IME, dtype: int64

In [11]:
gdf.VRSTA_ID.value_counts()

1    384159
2     92391
4      3396
3       368
Name: VRSTA_ID, dtype: int64

In [12]:
gdf[(gdf.VRSTA_ID == "2") & (gdf.POTEK_ID == "2")].VRSTA_ID.value_counts()

2    17681
Name: VRSTA_ID, dtype: int64

In [13]:
gdf[gdf.POTEK_ID == "2"].VRSTA_IME.value_counts()

prehod                   17681
tekoča voda                930
stoječa voda povezava       51
Name: VRSTA_IME, dtype: int64

Decision: Since most of the `"potek ni znan"` are calssified as `"prehod"`, we can just keep all of these features. The `"prehod"` features are important for breaching step in DEM pre-processing.

### Test TIPTV

In [14]:
gdf.TIPTV_IM.value_counts()

vodotok                                362697
melioracijski jarek                     75754
obcestni jarek                          33316
razbremenilni jarek                      8329
objekt (kanal) za posebno rabo vode       174
padavinski jarek                           44
Name: TIPTV_IM, dtype: int64

In [15]:
gdf.TIPTV_ID.value_counts()

1    362697
5     75754
4     33316
6      8329
7       174
8        44
Name: TIPTV_ID, dtype: int64

In [16]:
gdf[gdf.TIPTV_ID == "1"].VRSTA_IME.value_counts()

tekoča voda              294373
prehod                    65277
stoječa voda povezava      2679
akumulacijsko jezero        368
Name: VRSTA_IME, dtype: int64

In [17]:
# Save only "`"objekt (kanal) za posebno rabo vode"
tiptv = gdf[gdf.TIPTV_ID == "7"]
pth_tiptv = "c:\\Users\\ncoz\\ARRS_susa\\TIPTV"
os.makedirs(pth_tiptv, exist_ok=True)
tiptv.to_file(os.path.join(pth_tiptv, "tiptv_kanal-objekt.shp"))

Decision: include only IDs `"1": "vodotok"` and `"5": "melioracijski jarek"`.

### Test STANJE_IM

In [18]:
gdf.STANJE_IM.value_counts()

občasen         272665
stalen          207605
ni podatka          42
funkcionalen         1
suh                  1
Name: STANJE_IM, dtype: int64

In [19]:
gdf.STANJE_ID.value_counts()

3       272665
4       207605
9999        42
2            1
1            1
Name: STANJE_ID, dtype: int64

Decision: iclude only ID `"4": "stalen"`.

### Conclusion

In conclusion, we only filter by two criteria:

    - TIPTV == ("1" - vodotok) and ("5" - melioracije)
    - STANJE == ("4" - stalen)
    
EDIT 16/3/2021
Also add ("6" - razbremenilni jarek) - Gubarjev prekop in drugi

In [20]:
keep_cols = [
    "OBJECTID_1",
    "TIPTV_ID", "TIPTV_IM",
    "STANJE_ID", "STANJE_IM",
    "IME_ID", "IME",
    "VRSTA_ID", "VRSTA_IME",
    "geometry",
]
gdf = gdf[keep_cols]
gdf.tail(2)

,OBJECTID_1,TIPTV_ID,TIPTV_IM,STANJE_ID,STANJE_IM,IME_ID,IME,VRSTA_ID,VRSTA_IME,geometry
480312,15970,1,vodotok,3,občasen,0,0000 (zemljepisno ime v viru ni določeno),2,prehod,"LINESTRING Z (547845.940 116738.440 219.140, 5..."
480313,15971,1,vodotok,3,občasen,0,0000 (zemljepisno ime v viru ni določeno),2,prehod,"LINESTRING Z (546318.440 127203.610 692.370, 5..."


## Filter
Only select streams with:

    - TIPTV == ("1" - vodotok) and ("5" - melioracije) and ("6" - razbremenilni jarek)
    - STANJE == ("4" - stalen)

In [21]:
filtered = gdf[(gdf.TIPTV_ID.isin(["1", "5", "6"])) & (gdf.STANJE_ID == "4")]
filtered.tail(2)

,OBJECTID_1,TIPTV_ID,TIPTV_IM,STANJE_ID,STANJE_IM,IME_ID,IME,VRSTA_ID,VRSTA_IME,geometry
476307,21931,1,vodotok,4,stalen,0,0000 (zemljepisno ime v viru ni določeno),2,prehod,"LINESTRING Z (534309.280 134630.090 319.370, 5..."
476308,22962,1,vodotok,4,stalen,0,0000 (zemljepisno ime v viru ni določeno),2,prehod,"LINESTRING Z (588983.590 162427.830 182.070, 5..."


In [22]:
print(filtered.shape[0])
diff = gdf.shape[0] - filtered.shape[0]
print(f"Removed {diff} features (of {gdf.shape[0]} total).")

205523
Removed 274791 features (of 480314 total).


In [23]:
# Save filtered
pth_save = shp_src[:-4] + "_filter_tiptv_1_5_6_stanje_4.shp"
filtered.to_file(pth_save)